In [79]:
from jinja2 import Environment, FileSystemLoader
import os
import openai
import boto3
import json

def prompt_iterator(template: str):
  data_dir = './data'

  file_list = os.listdir(data_dir)
  file_list.sort(key=lambda x: int(x[1:x.index(".")]))

  for file_name in file_list:
    file_path = os.path.join(data_dir, file_name)

    with open(file_path, 'r') as file:
      data = file.read()

    output = template.render(data=data)
    
    yield output

env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('prompt.j2')

openai_client = openai.OpenAI()
def generate_openai_completion(prompt: str, test_case: int):
  response = openai_client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
      {"role": "assistant", "content": prompt},
    ],
    temperature=0.3,
  )

  completion = response.choices[0].message.content
  with open(f"./gpt4/d{test_case}.txt", 'w') as file:
    file.write(completion)


bedrock_client = boto3.client('bedrock-runtime', region_name='us-east-1')
def generate_claude_completion(prompt: str, test_case: int):
  body = {
    "messages": [{ "role": "user", "content": [{ "type": "text", "text": prompt}]}],
    "anthropic_version": "bedrock-2023-05-31",
    "temperature": 0.3,
    "top_p": 1,
    "top_k": 50,
    "max_tokens": 4096,
    }
  
  response = bedrock_client.invoke_model(modelId="anthropic.claude-3-sonnet-20240229-v1:0", body=json.dumps(body))

  response_body = json.loads(response["body"].read())
  completion = response_body["content"][0]["text"]

  with open(f"./claude3/d{test_case}.txt", 'w') as file:
    file.write(completion)

In [80]:
for i, p in enumerate(prompt_iterator(template)):
  if i == 0:
    generate_claude_completion(p, i + 1)